In [ ]:
!pip uninstall numpy -y
!pip uninstall torch torchvision torchaudio -y
!pip uninstall ultralytics -y
!pip uninstall accelerate -y
!pip uninstall ultralytics-thop -y
!pip uninstall xformers -y

!pip install numpy==1.26.4
!pip install --upgrade diffusers
!pip install --upgrade transformers

In [ ]:
conda install pytorch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0  pytorch-cuda=11.8 -c pytorch -c nvidia

In [ ]:
!pip install --upgrade accelerate
!pip install --upgrade safetensors
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
import torchvision
import torchaudio
import xformers

print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)
print(torch.cuda.is_available())
print(xformers.__version__)

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from pathlib import Path
from tqdm import tqdm
import random
import shutil
huggingface_token = "INPUT_YOUR_TOKEN"
pipe = StableDiffusionPipeline.from_pretrained(
    "Lykon/dreamshaper-8",
    torch_dtype=torch.float16,
    use_auth_token=huggingface_token,
    use_safetensors=True
).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
output_dir = Path("mask_images")
image_size = (640,640)
batch_size = 4
images_per_class = 500
num_inference_steps = 25

genders = [
    "asian man", "asian woman", "white man", "white woman",
    "black man", "black woman", "latino man", "latino woman",
    "middle eastern man", "middle eastern woman"
]
poses = [
    "front view",
    "side view",
    "three-quarter view",
    "looking up",
    "looking down",
    "head tilted left",
    "head tilted right"
]
backgrounds = [
    "indoor", "outdoor", "office", "street", "cafe",
    "school", "subway", "park", "shopping mall",
    "restaurant", "library"
]
clothes = [
    "in casual clothes", "wearing a hoodie", "in a business suit",
    "in a school uniform", "wearing sportswear", "wearing traditional clothes"
]
mask_colors = ["white", "beige", "blue", "black", "gray", "pink"]
expressions = [
    "smiling",
    "neutral expression",
    "frowning",
    "surprised expression",
    "serious expression",
    "happy expression"
]
shot_types = [
    "close-up face",             # 얼굴 부분만 가까이 찍은 사진입니다.
    "portrait, head and shoulders", # 얼굴과 어깨 부분까지 나타낸 초상화 스타일 사진입니다.
    "upper body portrait",       # 상체(가슴 위쪽)까지 나타낸 사진입니다.
    "full body portrait"         # 전신(머리부터 발끝까지)을 나타낸 사진입니다.
]
class_prompt_templates = {
    # 클래스 이름 'mask_on': 마스크를 착용한 얼굴 이미지를 생성하기 위한 지시문(프롬프트)입니다.
    "mask_on": (
        "a {shot} of a {gender} wearing a {mask_color} face mask covering mouth and nose properly, "
        "{expression}, {pose}, {background}, {clothes}, realistic, high-quality"
    ),

    # 클래스 이름 'no_mask': 마스크를 착용하지 않은 얼굴 이미지를 생성하기 위한 지시문입니다.
    "no_mask": (
        "a {shot} of a {gender} without mask, {expression}, {pose}, "
        "{background}, {clothes}, realistic, high-quality"
    )
}

negative_prompts = {
    "mask_on": "blurry, deformed face, low quality, unnatural skin",
    "no_mask": "blurry, deformed face, low quality, unnatural skin"
}

In [ ]:
for class_name, template in class_prompt_templates.items():
    class_folder = output_dir / class_name
    class_folder.mkdir(parents=True, exist_ok=True)
    for i in tqdm(range(0, images_per_class, batch_size), desc = class_name):
        prompts, negs = [], [negative_prompts[class_name]] * batch_size
        for _ in range(batch_size):
            prompt = template.format(
                shot = random.choice(shot_types),
                gender = random.choice(genders),
                pose = random.choice(poses),
                background = random.choice(backgrounds),
                clothes = random.choice(clothes),
                mask_color = random.choice(mask_colors) if class_name != "no_mask" else "",
                expression = random.choice(expressions)
            )
            prompts.append(prompt)
            
        # torch.inference_mode(): AI모델이 추론(실제 사용)만 할 때 쓰는 설정입니다.
        # torch.autocast("cuda"): GPU 연산을 더 효율적으로 수행하도록 자동으로 데이터 형태를 조정합니다.
        with torch.inference_mode(), torch.autocast("cuda"):

            # pipe(): 이미지 생성 AI 모델인 Stable Diffusion을 이용해 실제로 이미지를 생성하는 함수입니다.
            # prompts: 이미지 생성 시에 사용할 문장들의 리스트입니다.
            # negative_prompt: 생성 시 피하고 싶은 부정 특징을 담은 문장 리스트입니다.
            # height, width: 생성 이미지의 크기 (높이, 너비)
            # num_inference_steps: 하나의 이미지 생성 시 몇 단계를 반복할지
            images = pipe(
                prompts,
                negative_prompt=negs,
                height=image_size[1],
                width=image_size[0],
                num_inference_steps=num_inference_steps
            ).images

        # 생성된 이미지를 저장합니다.
        for idx, img in enumerate(images):
            # img.save(): 생성된 이미지를 저장하는 함수입니다.
            # f"{class_name}_{i+idx:05}.png": 이미지 파일 이름 형식 지정 (00001, 00002 등)
            img.save(class_folder / f"{class_name}_{i+idx:05}.png")